In [41]:
import os
import sys 
sys.path.append(os.path.abspath('..'))
import json
from pathlib import Path
from get_utils import get_data,get_dir
from model import test
import plot as plot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import seaborn as sb

#%matplotlib widget
%matplotlib qt

In [47]:
T = pd.read_csv(f'simulations/sim_prop_noise.csv')
#colors=[(0.5,0,0),(1,0.4,0.4),(0.8,0.8,0.2),(0,0,0)]
#colors=[(0.5,0,0),(0,0,0)]

fig,ax = plt.subplots(1,1,figsize=(10,5))
ax.set_ylabel('position loss')
ax.set_xlabel('perturbation time')
ax.set_title('proprioceptive perturbation') # proprioceptive
sb.lineplot(data=T,x='t',y='loss',hue='phase',style='noise') # ,palette=colors


<Axes: title={'center': 'proprioceptive perturbation'}, xlabel='perturbation time', ylabel='position loss'>